In [1]:
# Find car to cluster mapping

import json
from pytorch3d.io import load_obj
import torch

cluster_vert = torch.zeros(4,1352,3)
for k in range(4):
    vert, _, _ = load_obj(f'../apollo_mean_car_shape/merge_mean_car_model_{k}.obj')
    cluster_vert[k] = vert
    
id_to_cluster = {}
c = []
for car in range(79):
    vert, _, _ = load_obj(f'{car}.obj')
    dis = torch.abs(cluster_vert - vert).sum(-1).sum(-1)
    pesudo_k = torch.argmin(dis)
    c.append(int(pesudo_k))
    id_to_cluster[car] = int(pesudo_k)

with open('cluster_map.json','w') as f:
    json.dump(id_to_cluster,f)

In [9]:
# find use class
val = '/data/apollo/val/car_poses_labels'
import os, json

classes = set([])
for file in os.listdir(val):
    labels = json.load(open(f'{val}/{file}'))
    for l in labels:
        classes.add(l['car_id'])

In [12]:
classes

{2,
 6,
 7,
 8,
 9,
 12,
 14,
 16,
 18,
 19,
 20,
 23,
 25,
 27,
 28,
 31,
 32,
 35,
 37,
 40,
 43,
 46,
 47,
 48,
 50,
 51,
 54,
 56,
 60,
 61,
 66,
 70,
 71,
 76}

In [17]:
# Calculate mean shape and PCA matrix for whole obj

save = '../apollo_mean_car_shape'
N = 30

from pytorch3d.io import load_obj, save_obj
import os 
import numpy as np
import torch

# call whole meshesc
M = torch.zeros(79,1352,3)
for car in range(79):
    vert, face, _ = load_obj(f'{car}.obj')
    M[car] = vert
    
# compute mean shape
mean = M.mean(0)
# save mean shape
save_obj(os.path.join(save, 'mean.obj'), verts=mean, faces=face.verts_idx)

from sklearn.decomposition import PCA

# subtract mean shape
M_hat = M - mean

# compute PCA matrix
pca = PCA(n_components=N)
pca.fit(M_hat.reshape(79,-1))
S = pca.components_.reshape(-1,1352,3)

# save results
np.save(os.path.join(save, 'pca.npy'), S)

In [9]:
# Calculate mean shape and PCA matrix for reducted obj

save = '../apollo_mean_car_shape'
N = 20

from pytorch3d.io import load_obj, save_obj
import os 
import numpy as np
import torch
import json

with open('id_to_abb.json') as f:
    id_to_abb = json.load(f)

reducted_id = set()
for id, abb in id_to_abb.items():
    reducted_id.add(int(id))

# call whole meshes
M = torch.zeros(len(reducted_id),1352,3)
for i, car in enumerate(reducted_id):
    vert, face, _ = load_obj(f'{car}.obj')
    M[i] = vert
    
mean = M.mean(0)
# save mean shape
save_obj(os.path.join(save, 'red_mean.obj'), verts=mean, faces=face.verts_idx)

from sklearn.decomposition import PCA

# subtract mean shape
M_hat = M - mean

# compute PCA matrix
pca = PCA(n_components=N)
pca.fit(M_hat.reshape(len(reducted_id),-1))
S = pca.components_.reshape(-1,1352,3)

# save results
np.save(os.path.join(save, 'red_pca.npy'), S)

In [970]:
# Cluster new pca
save = '../apollo_mean_car_shape'

from pytorch3d.io import load_obj, save_obj
import os 
import numpy as np
import torch
import json
from sklearn.decomposition import PCA

K1 = [2,6,66,48]
K2 = [12,14,7]
K3 = [61,9]
K4 = [70,71,51,46,50,56,47,76,60,8,54]
K5 = [40,28,31,35,37,16,18,25,23,27,43,32,20,19]
cluster_map = {}
for id in range(79):
    if id in K1:
        cluster_map[id] = 0
    elif id in K2:
        cluster_map[id] = 1
    elif id in K3:
        cluster_map[id] = 2
    elif id in K4:
        cluster_map[id] = 3
    elif id in K5:
        cluster_map[id] = 4

with open('cluster_map.json','w') as f:
    json.dump(cluster_map,f)
    
K = K1 + K2 + K3+ K4+ K5

M1 = []
M2 = []
M3 = []
M4 = []
M5 = []

for car in K:
    vert, face, _ = load_obj(f'{car}.obj')
    if car in K1: M1.append(vert)
    elif car in K2: M2.append(vert)
    elif car in K3: M3.append(vert)
    elif car in K4: M4.append(vert)
    elif car in K5: M5.append(vert)
    
M1 = torch.stack(M1)
M2 = torch.stack(M2)
M3 = torch.stack(M3)
M4 = torch.stack(M4)
M5 = torch.stack(M5)
M = [M1,M2,M3,M4,M5]

for i, m in enumerate(M):
    mean= m.mean(0)
    save_obj(os.path.join(save, f'new_mean_{i}.obj'), verts=mean, faces=face.verts_idx)

In [55]:
# Test STD

save = '../apollo_mean_car_shape'
std = -4
pc = 2

from pytorch3d.io import load_obj, save_obj
import os 
import numpy as np
import torch

# call mean meshes and PCA
M, face, _ = load_obj(os.path.join(save, 'mean.obj'))
S = np.load(os.path.join(save, 'pca.npy'))[:10].reshape(10,-1)

b = np.array([0,0,0,0,0,0,0,0,0,0])
b[pc-1] = std

S_ = b @ S

M += S_.reshape(1352,3)
save_obj(f'pca_test/pc{pc}_{std}.obj', verts=M, faces=face.verts_idx)

In [212]:
# segment part
import renderer.render_egl as render

indices = range(100,200)
remap = {k:i for i,k in enumerate(indices)}

from pytorch3d.io import load_obj, save_obj
import os 
import numpy as np
import torch
import tqdm

# call mean meshes
M, face, _ = load_obj(os.path.join(save, 'mean.obj'))
M = M[indices]
F = face.verts_idx

f_mask = [True] * len(F)
# call segment faces
for i, f in tqdm.tqdm(enumerate(F),total= len(F)):
    for f_ in f:
        if f_ not in indices:
            f_mask[i] = False
            break
F = F[f_mask]
for f in F:
    for i in range(len(f)):
        f[i] = remap[int(f[i])]
        

save_obj(f'segment/test2.obj', verts=M, faces=F)

100%|█████████████████████████████████████| 2700/2700 [00:02<00:00, 1185.68it/s]


In [7]:
# Segment part, load def
import renderer.render_egl as render
import math
import numpy as np
import torch
import tqdm

def render_car(pose, vertices, face):
    """Render a car instance given pose and car_name
    """
    scale = np.ones((3, ))
    pose = np.array(pose)
    vert = project(pose, scale, vertices)
    intrinsic = np.float64([1024, 1024, 512, 512])
    depth, mask = render.renderMesh_py(np.float64(vert),
                                        np.float64(face),
                                        intrinsic,
                                        1024,
                                        1024,
                                        np.float64(0))

    return mask


def project(pose, scale, vertices):
    """ transform the vertices of a 3D car model based on labelled pose
    Input:
        pose: 0-3 rotation, 4-6 translation
        scale: the scale at each axis of the car
        vertices: the vertices position
    """

    if np.ndim(pose) == 1:
        mat = trans_vec_to_mat(pose[:3], pose[3:])
    elif np.ndim(pose) == 2:
        mat = pose

    vertices = vertices * scale
    p_num = vertices.shape[0]

    points = vertices.copy()
    points = np.hstack([points, np.ones((p_num, 1))])
    points = np.matmul(points, mat.transpose())

    return points[:, :3]


def trans_vec_to_mat(rot, trans, dim=4):
    """ project vetices based on extrinsic parameters to 3x4 matrix
    """
    mat = euler_angles_to_rotation_matrix(rot)
    mat = np.hstack([mat, trans.reshape((3, 1))])
    if dim == 4:
        mat = np.vstack([mat, np.array([0, 0, 0, 1])])

    return mat

def euler_angles_to_rotation_matrix(angle, is_dir=False):
    """Convert euler angels to quaternions.
    Input:
        angle: [roll, pitch, yaw]
        is_dir: whether just use the 2d direction on a map
    """
    roll, pitch, yaw = angle[0], angle[1], angle[2]
    rollMatrix = np.matrix([
        [1, 0, 0],
        [0, math.cos(roll), -math.sin(roll)],
        [0, math.sin(roll), math.cos(roll)]])

    pitchMatrix = np.matrix([
        [math.cos(pitch), 0, math.sin(pitch)],
        [0, 1, 0],
        [-math.sin(pitch), 0, math.cos(pitch)]])

    yawMatrix = np.matrix([
        [math.cos(yaw), -math.sin(yaw), 0],
        [math.sin(yaw), math.cos(yaw), 0],
        [0, 0, 1]])

    R = yawMatrix * pitchMatrix * rollMatrix
    R = np.array(R)

    if is_dir:
        R = R[:, 2]

    return R

In [972]:
# FIND TOP


indices = set(range(1000,1352)) - set([1004,1008,1003,1064,1063,1068,1117,1116,1008,1006,1066,1010,1012,1011,1070,1167,1118,1068,1006,1066,1007,1001,1000,1061,1067,1118,1121,1168,1174,1122,1030,1028,1087,1088,1173,1214,1168,1121,1118,1067,1061])
indices -= set([1062,1009,1068,1008,1068,1123,1028,1027,1086,1168,1118,1141,1138,1220,1172,1170,1120,1118,1060,1114,1068,1002,1113,1068,1166,1168,1163,1112,1087,1138,1086])
indices -= set([1065,1005,1069,1119,1169,1029,1085,1139,1215,1165,1059,1058,1057,1013,1111,1071,1014,1115,1022,1081,1021,1023,1192,1255,1216,1171,1219,1238,1191,1140,1135,1133,1180,1079,1024,1025,1134,1184,1185,1190,1237,1276,1256,1219,1217,1218,1026,1020,1083,1082,1132,1134,1184])
indices -= set([1080,1189,1213,1211,1164,1162,1137,1136,1183,1229,1230,1231,1236,1275,1274,1290,1258])
indices.update(set(range(700,1000)))
indices -= set([737,743,745,966,965,963,964,957,958,959,960,961,897,829,764,758,763,762,765,768,766,833,831,827,828,829,824,759,830,896,895,894,893,898,899,832,900,749,767,783,900,834,703])
indices -= set([901,814,748,947,948,748,863,944,738,744,806,740,752,754,730,877,811,880,940,805,874,742,739,805,808,750,246,878,])
indices -= set([946,943,820,949,997,817,799,733,837,802,937,736,865,930,884,812,872,732,935,800,815,934,868,883,])
indices -= set([751,871,931,760,826,892,772,700,769,709,838,970,950,818,998,836,734,870,942,876,804,810,882,746,936,798,962,956,704,702,770,710,708])
indices -= set([996,816,932,864,938,866,753,755,819,885,887,933,999,731,797,873,939,741,941,809,881,891,825,760,701,771,905,711,879,943,945,761,735])
indices -= set([813,747,803,867,801,869,807,875])
indices.update(set(range(500,700)))
indices -= set([504,568,636,692,558,624,690,626,560,562,628,694,696,632,564,548,546,506,570,510,636,502,500,566,634,514,580,622,556,620,554,516,582,518,648,688,622,550,530,596,610,678,544,542,598,664,552,682,616,662,532,612,698,680,630,614,600,618,676,686,684,666,608,674,])
indices -= set([672,540,604,536,668,508,572,640,670,602,538,534,606,642,574,627,693,569,505,503,501,561,625,691,559,695,629,623,557,699,547,545,679,611,543,677,609,541,551,537,535,637,507])
indices -= set([641,535,563,617,575,635,509,515,581,689,647,717,756,757,821,886,952,1124,683])
indices -= set([665,539,671,605,599,533,1289,1084,1186,1269,1268,1259,1306,1253,1212,687,621,555,513,579,517,631,675,687,663,685,643,573,567,565,633,619,601,667,549,553,531,597,615,673,607,603,669,613,697,681])

indices.update(set(range(200,500)))
indices -= set([293,413,479,419,245,239,299,257,311,407,473,347,353,203,365,497,383,281,317,449,227,341,377,263,431,287,233,359,251,305,425,221,491,329,275,389,269,383317,441,379,313,371,259,209,371,435,443,437,373,315,261,211,215,375,445,261,215,267,323,381,421,485,403,467,357,303,470,406,410,474,302,306,238,290,414,478,350,346,286,234,342,466,358,422,426,486,307,354,242,446,318,494,206,434,498,298,418,482,222,274,226,230,282,274,338,390,398,330,386,278,454,362,250,294,405,450,490,462,446,326,270,218,382,438,374,262,442,314,322,210,378,266,214,310,202,254,366,430,258,370,246,348,296,384,292,484,356,492,364,252,204,308,480,416,244,412,476,428,402,268,324,284,412,296,352,380,440,320,264,212,376,208,260,372,436,316,444,384,300,248,360,488,420,468,280,232,344,432,496,472,408,256,312,368,448,452,200,288,224,424,276,220,272,328,388,332,304,216,228,462,388,404,240,236])
indices -= set([481,201,493,499,433,483,295,241,297,415,417,369,416,265,253,243,298,429,355,321,367,309,255,205,322,255,266,296,471,361,409,291,391,327,475,489,411,469,345,343,289,235,349,351,427,495,439,447,385,387,451,271,301,217,331,285,311,453,225,247,249,277,219,271217,223,273,301,363,487,423,227,237,213,319,207,477,333,231,325,229])

indices.update(set(range(0,200)))
indices -= set([172,171,170,141,184,180,179,183,182,181,137,138,139,178,100,101,102,136,175,94,93,70,68,69,99,133,174,128,95,94,166,165,126,92,63,62,61,44,43,42,169,131,130,129,103,140,67,96,132,173,176,177,
               89,17,91,35,33,85,123,1,7,3,9,19,39,23,15,31,59,53,83,119,127,57,87,163,146,190,154,142,114,110,106,74,46,198,186,66,98,86,90,122,194,150,78,162,82,50,38,6,58,18,34,5,68,90,22,14,2,10,30,54,26,158,118,62,76,108,48,148,156,116,104,144,72,28,112,52,64,40,24,80,8,4,12,32,36,56,167,159,125,134,135,97,160,168,164,124,20,16,120,192,188,88,60,84,196,152,0,
               193,149,109,145,143,187,185,151,107,191,195,111,105,147,117,121,47,153,161,65,199,27,49,77,157,11,41,21,113,55,25,23,3,13,45,29,189,51,79,81,37,197,155,115,73,71,75])
indices -= set([649,951,583,337,279,455])
indices.update([1079,1211,1025,1185,1275,1217,281,1083,1137,1229,1269,1289,1259,1253,1134,1190,1082,1230,1274,1290,1218,1213,1237,1183,1258,1201,1231,1186,1306,1026,1236,1276,1132,1020,1268,1184,1256,1080,1136,1024,1212,1084,1014,1071,1124,1162,1164,819,885,951,821,905,826,962,838,970,886,960,892,956,952,513,453,452,435,436,437,443,331,388,262,316,389,317,374,373,217,267,229,228,226,278,218,297,183,180,184,375,231,323,382,438,517,579,703,501,570,580,757,649,717,637,567,507,573,565,633,643,711,769,771,583,755,689,761,515,647,581,509,641,575,701,635,761,570,506,510,574,642,710,770,702,634,566,502,518,582,688,648,500,516,572,640,708,508,700,704,772,756,446,445,383,337,455,341,325,454,326,398,448,338,381,444,333,390,384,324,447,423,1281,269,227,285,279,282,230,270,284,268,280,232,216,783,1180,709,462,783,930,863,1138,836,837])
indices -= set([448,508,642,700,267,388,710,633,575,777,574,640,779,572,708,771,634,571,705,711,706,702,641,701,715,325,446,341,269,316,269446,769,384,383,567,444,509,382,447,436,507,635,573,401,570,381,826,671,761,637,374,180,510,643,709401,326,772,324,704,270,373,506,438,387,370,270,500,500,323,566,443,437,268,232,285,375,501,435,565,703,502,317,445,297,423,217,218,216,184,262])
indices.update([715,706,401,571,777,779,705,581,826,887])

# indices = set(range(1352)) - indices

import json
indices = list(indices)
with open('top_indices.json','w') as f:
    json.dump(indices,f)

save = '../apollo_mean_car_shape'

from pytorch3d.io import load_obj, save_obj
import os 
import cv2
import torch
import numpy as np

# call mean meshes
M, face, _ = load_obj('43.obj')
F = face.verts_idx
V = M[indices]
V =  torch.cat([V,torch.ones(len(V),1)],dim=1).numpy().T
K = np.array([
    [1024, 0, 512],
    [0, 1024, 512],
    [0, 0, 1]
])
font = cv2.FONT_HERSHEY_SIMPLEX

for i, r in enumerate([0, (1/2)*np.pi, np.pi, (3/2)*np.pi]):
    pose = np.array([0, r, 0, 0, 0, 3])
    mask = render_car(pose, M.numpy(), F+1)
    mask = np.stack([mask,mask,mask],axis=2)
    mask *= 255
    Rt = trans_vec_to_mat(pose[:3], pose[3:])
    V_ = (Rt@ V)[:3,:]
    V_ = K @ V_
    V_ = (V_[:2] / V_[2:]).T
    
    for ind, (x,y) in zip(indices, V_):
        cv2.putText(mask, str(ind),(int(x),int(y)), font, 0.3, color=(0,0,255))
    cv2.imwrite(f'segment/test{i}.png',mask)

In [1166]:
# FIND BODY


with open('top_indices.json') as f:
    top_indices = json.load(f)

indices = set()
indices = set(range(500,1352)) - set(top_indices)

indices -= set([643,641,633,575,573,509,701,769,711,635,567,565,507,566,710,642,634,574,510,640,708,508,700,572,581,503,625,558,559,560,624,690,626,628,758,759,760,761,772,557,770,702,704,703,639,636,569,505,504,503,506,570,770,562,771,691,692,561,568,627,693,637,502,500,501])
indices.update(set(range(200,500)))
indices -= set(top_indices)

indices -= set([279,226,227,280,232,285,341,216,268,495,439,377,440,441,442,378,378,380,208,209,214,212,264,266,210,313,322,317,259,320,379,260,314,371,443,217,211,435,
               254,206,262,326,306,444,364,432,496,384,448,324,372,202,376,318,204,324,316,436,308,428,252,258,492,312,256,368])
indices -= set([121,493,201,381,261,373,267,315,375,445,325,253,265,429,321,309,207,427,307,367,319,255,205,447,213,319,270,382,438,430,446,269,218,215,437,323,311,257,497,383,431,365,203,263,383,215430,310,246,434,370,494,374,366,])

indices.update(set(range(0,200)))
indices -= set(top_indices)
indices -= set([76,17,37,43,121,113,41,11,13,77,49,161,91,125,137,167,163,119,65,157,89,15,67,27,21,117,63,123,85,127,83,87,55,3,3,9,5,1,15,23,39,31,61,53,35,33,57,59,159,61,85,162,122,86,50,179,101,69,7,19,141,137,103,129,93,95,171,131,169,165,78,34,82,42,196,84,369,112,152,120,160,28,36,44,24,40,12,140,102,0,20,4,14,32,68,70,100,138,180,184,170,130,172,94,158,118,62,16,20,6,128,166,172,124,164,88,92,18,90,168,126,60,38,56,58,54,22,10,8,2])
indices.update([306])

indices -= set([195,150,546,352,151,482,149,81,192,79,356,564,194,298,193,698,74,71,696,52,299,48,484,297,45,47,46,632,111,30,699,548,245,242,355,498,29,109,563,417,108,132,73,148,72,483,26,765,98,547,418,66,629,64,80,433,25,419,51,244,630,354,75,499,243,97,139,828,96,623,631,681,614,692,181,959,221,958,831,174,894,695,896,225,829,136,175,827,1023,330,274,173,134,825,891,897,961,824,830,764,694,762,99,222,275,182,763,768,133,178,895])
indices = list(indices)

import json
with open('body_indices.v2.json','w') as f:
    json.dump(indices,f)

save = '../apollo_mean_car_shape'

from pytorch3d.io import load_obj, save_obj
import os 
import cv2
import torch
import numpy as np

# call mean meshes
M, face, _ = load_obj(os.path.join(save, 'mean.obj'))
F = face.verts_idx
V = M[indices]
V =  torch.cat([V,torch.ones(len(V),1)],dim=1).numpy().T
K = np.array([
    [1024, 0, 512],
    [0, 1024, 512],
    [0, 0, 1]
])
font = cv2.FONT_HERSHEY_SIMPLEX

for i, r in enumerate([0, (1/2)*np.pi, np.pi, (3/2)*np.pi]):
    pose = np.array([0, r, 0, 0, 0, 3])
    mask = render_car(pose, M.numpy(), F+1)
    mask = np.stack([mask,mask,mask],axis=2)
    mask *= 255
    Rt = trans_vec_to_mat(pose[:3], pose[3:])
    V_ = (Rt@ V)[:3,:]
    V_ = K @ V_
    V_ = (V_[:2] / V_[2:]).T
    
    for ind, (x,y) in zip(indices, V_):
        cv2.putText(mask, str(ind),(int(x),int(y)), font, 0.3, color=(0,0,255))
    cv2.imwrite(f'segment/test{i}.png',mask)

In [1167]:
# FIND Head 


with open('top_indices.json') as f:
    top_indices = json.load(f)

with open('body_indices.v2.json') as f:
    body_indices = json.load(f)

indices = set(range(0, 1352)) - set(top_indices)- set(body_indices)

indices = list(indices)
import json
with open('head_indices.v2.json','w') as f:
    json.dump(indices,f)

    
save = '../apollo_mean_car_shape'

from pytorch3d.io import load_obj, save_obj
import os 
import cv2
import torch
import numpy as np

# call mean meshes
M, face, _ = load_obj('43.obj')
F = face.verts_idx
V = M[indices]
V =  torch.cat([V,torch.ones(len(V),1)],dim=1).numpy().T
K = np.array([
    [1024, 0, 512],
    [0, 1024, 512],
    [0, 0, 1]
])
font = cv2.FONT_HERSHEY_SIMPLEX

for i, r in enumerate([0, (1/2)*np.pi, np.pi, (3/2)*np.pi]):
    pose = np.array([0, r, 0, 0, 0, 3])
    mask = render_car(pose, M.numpy(), F+1)
    mask = np.stack([mask,mask,mask],axis=2)
    mask *= 255
    Rt = trans_vec_to_mat(pose[:3], pose[3:])
    V_ = (Rt@ V)[:3,:]
    V_ = K @ V_
    V_ = (V_[:2] / V_[2:]).T
    
    for ind, (x,y) in zip(indices, V_):
        cv2.putText(mask, str(ind),(int(x),int(y)), font, 0.3, color=(0,0,255))
    cv2.imwrite(f'segment/test{i}.png',mask)

In [605]:
# segment part
import renderer.render_egl as render


remap = {k:i for i,k in enumerate(indices)}

from pytorch3d.io import load_obj, save_obj
import os 
import numpy as np
import torch
import tqdm

# call mean meshes
M, face, _ = load_obj(os.path.join(save, 'mean.obj'))
M = M[indices]
F = face.verts_idx

f_mask = [True] * len(F)
# call segment faces
for i, f in tqdm.tqdm(enumerate(F),total= len(F)):
    for f_ in f:
        if f_ not in indices:
            f_mask[i] = False
            break
F = F[f_mask]
for f in F:
    for i in range(len(f)):
        f[i] = remap[int(f[i])]
        

save_obj(f'segment/test2.obj', verts=M, faces=F)

100%|█████████████████████████████████████| 2700/2700 [00:02<00:00, 1193.26it/s]


In [1170]:
# Compute PCA per part

save = '../apollo_mean_car_shape'
N = 10

def call_faces(indices,F):
    remap = {k:i for i,k in enumerate(indices)}
    f_mask = [True] * len(F)
    # call segment faces
    for i, f in tqdm.tqdm(enumerate(F),total= len(F)):
        for f_ in f:
            if f_ not in indices:
                f_mask[i] = False
                break
    F = F[f_mask]
    for f in F:
        for i in range(len(f)):
            f[i] = remap[int(f[i])]
    return F

from pytorch3d.io import load_obj, save_obj
import os 
import numpy as np
import torch
import json

with open('top_indices.json') as f:
    top_indices = json.load(f)

with open('body_indices.v2.json') as f:
    body_indices = json.load(f)
    
with open('head_indices.v2.json') as f:
    head_indices = json.load(f)

with open('id_to_abb.json') as f:
    id_to_abb = json.load(f)

reducted_id = set()
for id, abb in id_to_abb.items():
    reducted_id.add(int(id))

# call whole meshes
top_M = torch.zeros(len(reducted_id),len(top_indices),3)
head_M = torch.zeros(len(reducted_id),len(head_indices),3)
body_M = torch.zeros(len(reducted_id),len(body_indices),3)

for i, car in enumerate(reducted_id):
    vert, face, _ = load_obj(f'{car}.obj')
    top_vert = vert[top_indices]
    body_vert = vert[body_indices]
    head_vert = vert[head_indices]
    
    top_M[i] = top_vert
    body_M[i] = body_vert
    head_M[i] = head_vert


top_mean = top_M.mean(0)
body_mean = body_M.mean(0)
head_mean = head_M.mean(0)

top_face = call_faces(top_indices,face.verts_idx)
body_face = call_faces(body_indices,face.verts_idx)
head_face = call_faces(head_indices,face.verts_idx)

# save mean shape
save_obj(os.path.join(save, 'top_mean.obj'), verts=top_mean, faces=top_face)
save_obj(os.path.join(save, 'body_mean.v2.obj'), verts=body_mean, faces=body_face)
save_obj(os.path.join(save, 'head_mean.v2.obj'), verts=head_mean, faces=head_face)

from sklearn.decomposition import PCA

# subtract mean shape
top_M_hat = top_M - top_mean
body_M_hat = body_M - body_mean
head_M_hat = head_M - head_mean

# compute PCA matrix
top_pca = PCA(n_components=N)
top_pca.fit(top_M_hat.reshape(len(top_M_hat),-1))
top_S = top_pca.components_.reshape(N,-1,3)

# compute PCA matrix
body_pca = PCA(n_components=N)
body_pca.fit(body_M_hat.reshape(len(body_M_hat),-1))
body_S = body_pca.components_.reshape(N,-1,3)

# compute PCA matrix
head_pca = PCA(n_components=N)
head_pca.fit(head_M_hat.reshape(len(head_M_hat),-1))
head_S = head_pca.components_.reshape(N,-1,3)


# save results
np.save(os.path.join(save, 'top_pca.npy'), top_S)
np.save(os.path.join(save, 'body_pca.v2.npy'), body_S)
np.save(os.path.join(save, 'head_pca.v2.npy'), head_S)


100%|██████████████████████████████████████| 2700/2700 [00:03<00:00, 784.75it/s]


In [1169]:
head_S.shape

(10, 401, 3)